In [2]:
#Import modules
import pandas as pd
from pathlib import Path
import requests
import json
from pprint import pprint

#Pull in API key from personal file
from api_keys import geoapify_key

ModuleNotFoundError: No module named 'api_keys'

In [ ]:
#Set the file path for the csv
file = Path('Datasets/poverty.csv')

In [ ]:
#Read this csv file
df = pd.read_csv(file, encoding="ISO-8859-1")

In [3]:
#Preview the dataframe
df.head(100)

NameError: name 'df' is not defined

In [5]:
# Identify incomplete rows
df.count()

country            4877
year               4877
reporting_level    4411
welfare_type       4411
ppp_version        4877
                   ... 
palma_ratio        4395
s80_s20_ratio      4395
p90_p10_ratio      4877
p90_p50_ratio      4877
p50_p10_ratio      4877
Length: 108, dtype: int64

In [6]:
#Information on the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4877 entries, 0 to 4876
Columns: 108 entries, country to p50_p10_ratio
dtypes: float64(103), int64(2), object(3)
memory usage: 4.0+ MB


In [7]:
# Revise dataframe to only have information we'll be using for analysis 
df = df[['country', 'year', 'reporting_level', 'welfare_type', 'headcount_ratio_international_povline', 'median']]
df.head(100)

,country,year,reporting_level,welfare_type,headcount_ratio_international_povline,median
0,Albania,1996,national,consumption,0.920669,5.774805
1,Albania,2002,national,consumption,1.570843,5.539607
2,Albania,2005,national,consumption,0.860527,6.460357
3,Albania,2008,national,consumption,0.313650,6.957659
4,Albania,2012,national,consumption,0.849754,6.825289
...,...,...,...,...,...,...
95,Austria,2007,national,income,0.372292,45.166239
96,Austria,2008,national,income,0.471167,46.720954
97,Austria,2009,national,income,0.656235,47.732060
98,Austria,2010,national,income,0.490703,48.601187


In [8]:
# Identify incomplete rows
df.count()

country                                  4877
year                                     4877
reporting_level                          4411
welfare_type                             4411
headcount_ratio_international_povline    4877
median                                   4877
dtype: int64

In [9]:
#Information on the revised dataframe
df.info

<bound method DataFrame.info of        country  year reporting_level welfare_type  \
0      Albania  1996        national  consumption   
1      Albania  2002        national  consumption   
2      Albania  2005        national  consumption   
3      Albania  2008        national  consumption   
4      Albania  2012        national  consumption   
...        ...   ...             ...          ...   
4872    Zambia  2010        national  consumption   
4873    Zambia  2015        national  consumption   
4874  Zimbabwe  2011        national  consumption   
4875  Zimbabwe  2017        national  consumption   
4876  Zimbabwe  2019        national  consumption   

      headcount_ratio_international_povline    median  
0                                  0.920669  5.774805  
1                                  1.570843  5.539607  
2                                  0.860527  6.460357  
3                                  0.313650  6.957659  
4                                  0.849754  6.8252

In [10]:
# Checking data types
df.dtypes

country                                   object
year                                       int64
reporting_level                           object
welfare_type                              object
headcount_ratio_international_povline    float64
median                                   float64
dtype: object

In [11]:
# Display an overview of the reporting_level column
df['reporting_level'].value_counts()

national    4049
urban        236
rural        126
Name: reporting_level, dtype: int64

In [12]:
# Display an overview of the welfare_type column
df['welfare_type'].value_counts()

income         2464
consumption    1947
Name: welfare_type, dtype: int64

In [13]:
# Determine max year of dataframe
max(df['year'])

2021

In [14]:
# Determine minimum year of dataframe
min(df['year'])

1967

In [15]:
# Display an overview of the country column
df['country'].value_counts()

Indonesia       174
China           120
Poland           92
Romania          74
India            72
               ... 
Suriname          2
Lebanon           2
Tuvalu            2
Turkmenistan      2
Somalia           2
Name: country, Length: 177, dtype: int64

In [17]:
#set parameters for later API searches on GeoApify
base_url = "https://api.geoapify.com/v1/geocode/search"
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "limit": 1
}

#Add columns for geoapify country name and iso_alpha to normalize across datasets
df['geo_country'] = ""
df['iso_two'] = ""

In [18]:
# Iterate dataset to remove non-countries and normalize name syntax and iso key
for index, row in df.iterrows():

    #Set country parameter for current row and call API
    params['country'] = df.loc[index, 'country']
    current_country = requests.get(base_url, params).json()
    
    # Attempt to add proper syntax to geo_country and iso_two in base dataframe
    try: 
        df.loc[index, 'geo_country'] = current_country['results'][0]['country']
        df.loc[index, 'iso_two'] = current_country['results'][0]['country_code']
        # Show progress for each loop
        print(f"Indexing: {df.loc[index, 'geo_country']} as {df.loc[index, 'iso_two']} and confidence: {current_country['results'][0]['rank']['confidence']}")

        #If the serach confidence is zero, drop the row
        if current_country['results'][0]['rank']['confidence'] == 0:
            df = df.drop(labels=index, axis=0)

    #If the above try fails with no country/iso, delete the row
    except IndexError:
        print(f"Dropping: {current_country}")
        df = df.drop(labels=index, axis=0)
        

Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Algeria as dz and confidence: 1
Indexing: Algeria as dz and confidence: 1
Indexing: Algeria as dz and confidence: 1
Indexing: Angola as ao and confidence: 1
Indexing: Angola as ao and confidence: 1
Indexing: Angola as ao and confidence: 1
Indexing: Argentina as ar and confidence: 1
Indexing: Argentina as ar and confidence: 1
Indexing: Argentina as ar and confidence: 1
Indexing: Argentina as ar and c

Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bolivia as bo and confidence: 1
Indexing: Bosnia and Herzegovina as ba and confidence: 1
Indexing: Bosnia and Herzegovina as ba and confidence: 1
Indexing: Bosnia and Herzegovina as ba and confidence: 1
Indexing: Bosnia and Herzegovina as ba and confidence: 1
Indexing: Botswana as bw and confidence: 1
Indexing: Botswana as bw and confidence: 1
Indexing: Bots

Indexing: China as cn and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colombia as co and confidence: 1
Indexing: Colo

Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'East Asia and Pacific', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'East Asia and Pacific', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'East Asia and Pacific', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'East Asia and Pacific', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'East Asia and Pacific', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'East Asia and Pacific', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'East Asia and Pacific', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'East Asia and Pacific', 'expected_type': 'unknown'}}}


Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'Europe and Central Asia', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'Europe and Central Asia', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'Europe and Central Asia', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'Europe and Central Asia', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'Europe and Central Asia', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'Europe and Central Asia', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'Europe and Central Asia', 'expected_type': 'unknown'}}}
Dropping: {'results': [], 'query': {'text': '', 'parsed': {'country': 'Europe and Central Asia', 'expected_type

...

Indexing: Pakistan as pk and confidence: 1
Indexing: Pakistan as pk and confidence: 1
Indexing: Pakistan as pk and confidence: 1
Indexing: Pakistan as pk and confidence: 1
Indexing: Palestinian Territory as ps and confidence: 1
Indexing: Palestinian Territory as ps and confidence: 1
Indexing: Palestinian Territory as ps and confidence: 1
Indexing: Palestinian Territory as ps and confidence: 1
Indexing: Palestinian Territory as ps and confidence: 1
Indexing: Palestinian Territory as ps and confidence: 1
Indexing: Palestinian Territory as ps and confidence: 1
Indexing: Palestinian Territory as ps and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: Panama as pa and confidence: 1
Indexing: P

Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Romania as ro and confidence: 1
Indexing: Russia as ru and confidence: 1
Indexing: Russia as ru and confidence: 1
Indexing: Russia as ru and confidence: 1
Indexing: Russia as ru and confidence: 1
Indexing: Russia as ru and confidence: 1
Indexing: Russia as ru and confidence: 1
Indexing: Russia as ru and confidence: 1

Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Spain as es and confidence: 1
Indexing: Sri Lanka as lk and confidence: 1
Indexing: Sri Lanka as lk and confidence: 1
Indexing: Sri Lanka as lk and confidence: 1
Indexing: Sri Lanka as lk and confidence: 1
Indexing: Sri Lanka as lk and confidence: 1
Indexing: Sri Lanka 

Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: East Timor as tl and confidence: 0
Indexing: East Timor as tl and confidence: 0
Indexing: East Timor as tl and confidence: 0
Indexing: Togo as tg and confidence: 1
Indexing: Togo as tg and confidence: 1
Indexing: Togo as tg and confidence: 1
Indexing: Togo as tg and confidence: 1
Indexing: Tonga as to and confidence: 1
Indexing: Tonga as to and confidence: 1
Indexing: Tonga as to and c

Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Venezuela as ve and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing: Vietnam as vn and confidence: 1
Indexing

In [19]:
# Display an overview of the country column
df['country'].value_counts()

Indonesia       174
China           120
Poland           92
Romania          74
India            72
               ... 
Lebanon           2
Suriname          2
Tuvalu            2
Turkmenistan      2
Nauru             2
Name: country, Length: 165, dtype: int64

In [20]:
# Preview the dataframe
df.head(100)

,country,year,reporting_level,welfare_type,headcount_ratio_international_povline,median,geo_country,iso_two
0,Albania,1996,national,consumption,0.920669,5.774805,Albania,al
1,Albania,2002,national,consumption,1.570843,5.539607,Albania,al
2,Albania,2005,national,consumption,0.860527,6.460357,Albania,al
3,Albania,2008,national,consumption,0.313650,6.957659,Albania,al
4,Albania,2012,national,consumption,0.849754,6.825289,Albania,al
...,...,...,...,...,...,...,...,...
95,Austria,2007,national,income,0.372292,45.166239,Austria,at
96,Austria,2008,national,income,0.471167,46.720954,Austria,at
97,Austria,2009,national,income,0.656235,47.732060,Austria,at
98,Austria,2010,national,income,0.490703,48.601187,Austria,at


In [21]:
# Delete the rows containing urban and rural reporting_level
df = df[df["reporting_level"].str.contains("urban|rural") == False]

In [22]:
# Display an overview of the reporting_level column
df['reporting_level'].value_counts()

national    4013
Name: reporting_level, dtype: int64

In [23]:
# Identify incomplete rows
df.count()

country                                  4013
year                                     4013
reporting_level                          4013
welfare_type                             4013
headcount_ratio_international_povline    4013
median                                   4013
geo_country                              4013
iso_two                                  4013
dtype: int64

In [25]:
# Print Cleaned dataframe
df

,country,year,reporting_level,welfare_type,headcount_ratio_international_povline,median,geo_country,iso_two
0,Albania,1996,national,consumption,0.920669,5.774805,Albania,al
1,Albania,2002,national,consumption,1.570843,5.539607,Albania,al
2,Albania,2005,national,consumption,0.860527,6.460357,Albania,al
3,Albania,2008,national,consumption,0.313650,6.957659,Albania,al
4,Albania,2012,national,consumption,0.849754,6.825289,Albania,al
...,...,...,...,...,...,...,...,...
4872,Zambia,2010,national,consumption,68.456606,1.353084,Zambia,zm
4873,Zambia,2015,national,consumption,61.352160,1.570981,Zambia,zm
4874,Zimbabwe,2011,national,consumption,21.580063,3.798868,Zimbabwe,zw
4875,Zimbabwe,2017,national,consumption,34.206046,2.874558,Zimbabwe,zw


In [1]:
df['year'].value_counts()

NameError: name 'df' is not defined